In [1]:
import gevent.monkey
gevent.monkey.patch_all(aggressive=False, thread=False)
from ocs import client_wampy as cw
import ocs
from ocs_widgets import TaskWidget, ProcessWidget

parser = ocs.site_config.add_arguments()

args = parser.parse_args([])
ocs.site_config.reparse_args(args, '*host*')
site, host, _ = ocs.site_config.get_config(args, '*host*')
root = site.hub.data['address_root']

instance_ids = ['thermo1', 'aggregator']
clients = {i: cw.ControlClient(
        "{}.{}".format(root, i),
        url=site.hub.data['wamp_server'],
        realm=site.hub.data['wamp_realm'])
    for i in instance_ids
}
for _,c in clients.items():
    c.start()

/Users/jacoblashner/Library/Python/3.7/lib/python/site-packages/wampy/__init__.py:28: MonkeyPatchWarning: Patching more than once will result in the union of all True parameters being patched
  gevent.monkey.patch_all(thread=False)


In [2]:
c = clients['thermo1']
params = {}
therm_init = TaskWidget(clients['thermo1'], 'init_lakeshore', params)
display(therm_init)

params = {}
acq = ProcessWidget(clients['thermo1'], 'acq', params)
display(acq)

c = clients['aggregator']
params = {}
therm_init = TaskWidget(c, 'initialize', params)
display(therm_init)

params = {
    "time_per_file": 10 * 60,
    "data_dir": "/data/hk/"
}
acq = ProcessWidget(c, 'record', params)
display(acq)

TaskWidget(children=(VBox(children=(Label(value='observatory.thermo1: init_lakeshore'), Button(description='St…

ProcessWidget(children=(VBox(children=(Label(value='observatory.thermo1: acq'), HBox(children=(Button(descript…

TaskWidget(children=(VBox(children=(Label(value='observatory.aggregator: initialize'), Button(description='Sta…

ProcessWidget(children=(VBox(children=(Label(value='observatory.aggregator: record'), HBox(children=(Button(de…